<a href="https://colab.research.google.com/github/gurunars/ipythons/blob/main/homm_creatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import requests
import json
from bs4 import BeautifulSoup

ROOT = "https://heroes.thelazy.net"

PATH = ROOT + "/index.php/List_of_creatures_(HotA)"

def get_creatures():
  html_doc = requests.get(PATH).text

  soup = BeautifulSoup(html_doc, 'html.parser')

  rows = soup.select('tr')

  for row in rows:
    cols = row.select('td')
    if not cols:
      continue

    yield dict(
      icon_url = ROOT + cols[0].select('img')[0].attrs['src'].strip(),
      castle_url = ROOT + cols[1].select('img')[0].attrs['src'].strip(),
      name = cols[0].select('a')[-1].text.strip(),
      hp = int(cols[7].select('span')[0].text.strip()),
      cost = int(cols[11].text.strip())
    )

print(json.dumps(list(get_creatures()), indent=2))


[
  {
    "icon_url": "https://heroes.thelazy.net/images/3/30/Creature_portrait_Nymph_small.gif",
    "castle_url": "https://heroes.thelazy.net/images/4/4a/Town_portrait_Cove_small.gif",
    "name": "Nymph",
    "hp": 4,
    "cost": 35
  },
  {
    "icon_url": "https://heroes.thelazy.net/images/5/56/Creature_portrait_Oceanid_small.gif",
    "castle_url": "https://heroes.thelazy.net/images/4/4a/Town_portrait_Cove_small.gif",
    "name": "Oceanid",
    "hp": 4,
    "cost": 45
  },
  {
    "icon_url": "https://heroes.thelazy.net/images/b/be/Creature_portrait_Crew_Mate_small.gif",
    "castle_url": "https://heroes.thelazy.net/images/4/4a/Town_portrait_Cove_small.gif",
    "name": "Crew Mate",
    "hp": 15,
    "cost": 110
  },
  {
    "icon_url": "https://heroes.thelazy.net/images/1/1e/Creature_portrait_Seaman_small.gif",
    "castle_url": "https://heroes.thelazy.net/images/4/4a/Town_portrait_Cove_small.gif",
    "name": "Seaman",
    "hp": 15,
    "cost": 140
  },
  {
    "icon_url": "htt